In [1]:
include("bricks.jl")
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [2]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3))
f = @formula(y ~ 1 + a + b + c)
contrasts = Dict(:b=>ContinuousTerm, :c => CategoricalTerm)

Dict{Symbol, UnionAll} with 2 entries:
  :b => ContinuousTerm
  :c => CategoricalTerm

In [3]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:22


Main.HighDimMM.highDimMixedModel{Float64}(y ~ 1 + a + b + c, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([1.0; 2.0; … ; 8.0; 9.0]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([1.0 0.236781883208121; 1.0 0.9437409715735081; … ; 1.0 0.617492416565387; 1.0 0.48153065407402607]), [0.0 0.0; 1.0 0.0; … ; 1.0 0.0; 0.0 1.0], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

In [5]:
# Three matrix
HMM.M.M; HMM.X.X; HMM.Z.Z

9×2 Matrix{Float64}:
 0.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  0.0
 1.0  0.0
 0.0  1.0

In [6]:
# response
HMM.y

9-element Vector{Float64}:
 0.5851946422124186
 0.07733793456911231
 0.7166282400543453
 0.3203570514066232
 0.6530930076222579
 0.2366391513734556
 0.7096838914472361
 0.5577872440804086
 0.05079002172175784

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$

To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

In [12]:
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
C = transpose(u[:,1:6]) # simplify: assume fixed effect full rank
K = C*P

6×9 Matrix{Float64}:
 -9.13469e-18  -0.190676   -0.0463644  …  -0.672223    0.286397   -0.0154922
  0.721784     -0.199205   -0.349734      -0.0846755   0.256278    0.290553
  9.0029e-18   -0.517681    0.0175724      0.0823317   0.0976185  -0.314116
 -4.07539e-17   0.177124   -0.256386      -0.40938     0.0725489  -0.0347221
 -2.14464e-17  -0.0397892  -0.15056       -0.155654   -0.696412    0.579134
 -2.29003e-17  -0.292176    0.780011   …  -0.309793    0.0146294   0.270772

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$